# 📈 S&P 500 Sector Performance, Risk & 2026 Investment Outlook
**Author:** [Your Name] | **LinkedIn:** [Your URL] | **GitHub:** [Your URL]

---

## Project Overview
A full end-to-end data analysis of **30 S&P 500 stocks** across **6 sectors** from **2020–2025**, culminating in a **2026 sector investment forecast** using a linear trend model with bootstrap confidence intervals.

### Business Questions Answered
1. Which sectors delivered the best risk-adjusted returns over 5 years?
2. How does volatility correlate with returns — does more risk pay off?
3. Are there seasonal return patterns investors can exploit?
4. How correlated are stocks within vs. across sectors (diversification value)?
5. **Which sectors should investors OVERWEIGHT, NEUTRAL, or UNDERWEIGHT in 2026?**

### Tools & Skills Demonstrated
| Tool | Usage |
|---|---|
| `yfinance` | Real market data via API |
| `pandas` | Data wrangling, reshaping, feature engineering |
| `SQLite + SQL` | Querying 1500+ days × 30 stocks |
| `matplotlib + seaborn` | 9 publication-quality charts |
| `scikit-learn` | Linear regression + bootstrap CI for 2026 forecasts |
| `Tableau` | Interactive dashboard (exported CSVs included) |

> **Note:** Run Section 0 first to install dependencies, then run all cells top-to-bottom.


## Section 0 — Install Dependencies

In [ ]:
# Run this once, then restart kernel if needed
import subprocess, sys
pkgs = ['yfinance','pandas','numpy','matplotlib','seaborn','scikit-learn']
subprocess.check_call([sys.executable, '-m', 'pip', 'install', '--quiet'] + pkgs)
print('✅ All dependencies installed')

## Section 1 — Imports & Configuration

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import seaborn as sns
import yfinance as yf
import sqlite3, warnings, os
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, r2_score
from matplotlib.patches import Patch

warnings.filterwarnings('ignore')
sns.set_theme(style='whitegrid', palette='muted', font_scale=1.15)
plt.rcParams['figure.dpi'] = 120
TITLE_KW = dict(fontsize=15, fontweight='bold', pad=14)

os.makedirs('plots',  exist_ok=True)
os.makedirs('data',   exist_ok=True)

print('✅ Libraries loaded')

## Section 2 — Stock Universe & Data Download

In [ ]:
# ── Stock Universe ──────────────────────────────────────────────────────
SECTORS = {
    'Technology':  ['AAPL', 'MSFT', 'NVDA', 'AMD',  'INTC'],
    'Healthcare':  ['JNJ',  'PFE',  'UNH',  'ABBV', 'MRK'],
    'Finance':     ['JPM',  'BAC',  'GS',   'WFC',  'MS'],
    'Energy':      ['XOM',  'CVX',  'COP',  'SLB',  'EOG'],
    'Consumer':    ['AMZN', 'TSLA', 'HD',   'MCD',  'NKE'],
    'Utilities':   ['NEE',  'DUK',  'SO',   'D',    'EXC']
}
ALL_TICKERS   = [t for v in SECTORS.values() for t in v]
ticker_sector = {t: s for s, tickers in SECTORS.items() for t in tickers}

START_DATE = '2020-01-01'
END_DATE   = '2025-12-31'

print(f'Universe: {len(ALL_TICKERS)} stocks across {len(SECTORS)} sectors')
print(f'Period  : {START_DATE} → {END_DATE}')

# ── Download ─────────────────────────────────────────────────────────────
print('\nDownloading from Yahoo Finance (this takes ~30 seconds)...')
raw    = yf.download(ALL_TICKERS, start=START_DATE, end=END_DATE, auto_adjust=True, progress=True)
prices = raw['Close'].copy()
prices.index = pd.to_datetime(prices.index)

# Clean
prices = prices.ffill().dropna()
print(f'\n✅ Download complete')
print(f'   Shape: {prices.shape[0]} trading days × {prices.shape[1]} stocks')
print(f'   Missing values: {prices.isnull().sum().sum()}')
prices.tail(3)

## Section 3 — Feature Engineering

In [ ]:
# ── Daily Returns ──────────────────────────────────────────────────────
daily_returns = prices.pct_change().dropna()

# ── Long Format ─────────────────────────────────────────────────────────
returns_long = daily_returns.reset_index().melt(
    id_vars='Date', var_name='Ticker', value_name='DailyReturn')
returns_long['Sector']  = returns_long['Ticker'].map(ticker_sector)
returns_long['Year']    = returns_long['Date'].dt.year
returns_long['Month']   = returns_long['Date'].dt.month
returns_long['Quarter'] = returns_long['Date'].dt.quarter

# ── Stock Summary Stats ─────────────────────────────────────────────────
RISK_FREE = 0.04 / 252  # 4% annual risk-free rate

stock_summary = daily_returns.apply(lambda col: pd.Series({
    'ann_return': col.mean() * 252,
    'ann_vol':    col.std()  * np.sqrt(252),
    'sharpe':    (col.mean() - RISK_FREE) / col.std() * np.sqrt(252),
    'max_dd':     (col + 1).cumprod().div((col+1).cumprod().cummax()).min() - 1
})).T
stock_summary['Sector'] = stock_summary.index.map(ticker_sector)
stock_summary = stock_summary.reset_index().rename(columns={'index':'Ticker'})

sector_daily  = returns_long.groupby(['Date','Sector'])['DailyReturn'].mean().unstack()
sector_cumret = (1 + sector_daily).cumprod() - 1

palette       = sns.color_palette('tab10', n_colors=len(SECTORS))
sector_colors = dict(zip(SECTORS.keys(), palette))

print(f'✅ Feature engineering complete')
print(f'   Long-format rows : {len(returns_long):,}')
print(f'   Stocks summarised: {len(stock_summary)}')
stock_summary.sort_values('ann_return', ascending=False).head(10)

## Section 4 — SQL Analysis

In [ ]:
# ── Save to SQLite ──────────────────────────────────────────────────────
conn = sqlite3.connect('data/stock_analysis.db')
returns_long.to_sql('daily_returns', conn, if_exists='replace', index=False)
prices.reset_index().melt(id_vars='Date', var_name='Ticker', value_name='Close') \
      .assign(Sector=lambda df: df['Ticker'].map(ticker_sector)) \
      .to_sql('prices', conn, if_exists='replace', index=False)
print('✅ SQLite database created: data/stock_analysis.db')

# ── Q1: Top 10 stocks by annualised return ────────────────────────────────
print('\n=== Q1: Top 10 Stocks by Annualised Return ===')
q1 = pd.read_sql("""
    SELECT Ticker, Sector,
           ROUND(AVG(DailyReturn)*252*100, 2) AS ann_return_pct,
           COUNT(*) AS trading_days
    FROM daily_returns
    GROUP BY Ticker, Sector
    ORDER BY ann_return_pct DESC
    LIMIT 10
""", conn)
print(q1.to_string(index=False))

# ── Q2: Sector yearly performance ────────────────────────────────────────
sector_yearly = pd.read_sql("""
    SELECT Sector, Year,
           ROUND(SUM(DailyReturn)*100, 2) AS yearly_return_pct,
           ROUND(AVG(DailyReturn)*SQRT(252)*100, 2) AS ann_vol_pct
    FROM daily_returns
    GROUP BY Sector, Year
    ORDER BY Sector, Year
""", conn)

print('\n=== Q2: Sector Yearly Performance ===')
print(sector_yearly.pivot(index='Year', columns='Sector', values='yearly_return_pct').round(1).to_string())

# ── Q3: Worst 10 sector-month periods ────────────────────────────────────
print('\n=== Q3: 10 Worst Sector-Month Periods (Drawdown Risk) ===')
q3 = pd.read_sql("""
    SELECT Sector, Year, Month,
           ROUND(SUM(DailyReturn)*100, 2) AS monthly_return_pct
    FROM daily_returns
    GROUP BY Sector, Year, Month
    ORDER BY monthly_return_pct ASC
    LIMIT 10
""", conn)
print(q3.to_string(index=False))

## Section 5 — Exploratory Data Analysis (9 Charts)

In [ ]:
# ── Chart 1: Cumulative Returns by Sector ────────────────────────────────
fig, ax = plt.subplots(figsize=(14,6))
for col in sector_cumret.columns:
    ax.plot(sector_cumret.index, sector_cumret[col]*100, label=col, linewidth=2, color=sector_colors[col])
ax.axhline(0, color='black', linewidth=0.8, linestyle='--')
ax.yaxis.set_major_formatter(mtick.PercentFormatter())
ax.set_title('Cumulative Sector Returns (2020–2025)', **TITLE_KW)
ax.set_xlabel('Date'); ax.set_ylabel('Cumulative Return (%)')
ax.legend(title='Sector', bbox_to_anchor=(1.01,1), loc='upper left')
plt.tight_layout()
plt.savefig('plots/plot_01_cumulative_returns.png', bbox_inches='tight')
plt.show()
print('✅ Chart 1 saved')

In [ ]:
# ── Chart 2: Risk vs Return Scatter ──────────────────────────────────────
fig, ax = plt.subplots(figsize=(13,7))
for sector, grp in stock_summary.groupby('Sector'):
    ax.scatter(grp['ann_vol']*100, grp['ann_return']*100, label=sector, s=130,
               alpha=0.85, color=sector_colors[sector], edgecolors='white', linewidth=0.5)
    for _, row in grp.iterrows():
        ax.annotate(row['Ticker'], (row['ann_vol']*100, row['ann_return']*100),
                    textcoords='offset points', xytext=(6,2), fontsize=8, alpha=0.8)
ax.axhline(0, color='gray', linewidth=0.8, linestyle='--')
ax.xaxis.set_major_formatter(mtick.PercentFormatter())
ax.yaxis.set_major_formatter(mtick.PercentFormatter())
ax.set_title('Risk vs. Return — All 30 Stocks (Annualised, 2020–2025)', **TITLE_KW)
ax.set_xlabel('Annualised Volatility (%)'); ax.set_ylabel('Annualised Return (%)')
ax.legend(title='Sector', bbox_to_anchor=(1.01,1), loc='upper left')
plt.tight_layout()
plt.savefig('plots/plot_02_risk_return.png', bbox_inches='tight')
plt.show()
print('✅ Chart 2 saved')

In [ ]:
# ── Chart 3: Sharpe Ratio Bar ─────────────────────────────────────────────
sharpe_sector = stock_summary.groupby('Sector')['sharpe'].mean().sort_values(ascending=True)
fig, ax = plt.subplots(figsize=(10,5))
colors_b = ['#d73027' if v < 0 else '#4575b4' for v in sharpe_sector.values]
bars = ax.barh(sharpe_sector.index, sharpe_sector.values, color=colors_b, edgecolor='white', height=0.6)
ax.axvline(0, color='black', linewidth=0.8)
ax.bar_label(bars, fmt='%.2f', padding=5, fontsize=10)
ax.set_title('Average Sharpe Ratio by Sector (2020–2025)', **TITLE_KW)
ax.set_xlabel('Sharpe Ratio (Higher = Better Risk-Adjusted Return)')
plt.tight_layout()
plt.savefig('plots/plot_03_sharpe_ratio.png', bbox_inches='tight')
plt.show()
print('✅ Chart 3 saved')

In [ ]:
# ── Chart 4: Correlation Heatmap ──────────────────────────────────────────
corr    = daily_returns.corr()
ordered = [t for tickers in SECTORS.values() for t in tickers if t in corr.columns]
corr    = corr.loc[ordered, ordered]
fig, ax = plt.subplots(figsize=(14,11))
mask    = np.triu(np.ones_like(corr, dtype=bool))
sns.heatmap(corr, mask=mask, cmap='RdYlGn', center=0, vmin=-0.2, vmax=1,
            annot=False, linewidths=0.4, cbar_kws={'label':'Pearson Correlation'}, ax=ax)
ax.set_title('Return Correlation Matrix — 30 S&P 500 Stocks, Grouped by Sector (2020–2025)', **TITLE_KW)
plt.tight_layout()
plt.savefig('plots/plot_04_correlation_heatmap.png', bbox_inches='tight')
plt.show()
print('✅ Chart 4 saved')

In [ ]:
# ── Chart 5: Seasonality Heatmap ──────────────────────────────────────────
monthly = returns_long.groupby(['Year','Month'])['DailyReturn'].sum().unstack('Month') * 100
monthly.columns = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
fig, ax = plt.subplots(figsize=(14,5))
sns.heatmap(monthly, cmap='RdYlGn', center=0, annot=True, fmt='.1f',
            linewidths=0.5, cbar_kws={'label':'Monthly Return (%)'}, ax=ax)
ax.set_title('Market Seasonality — Monthly Returns (%) Across All Stocks (2020–2025)', **TITLE_KW)
plt.tight_layout()
plt.savefig('plots/plot_05_seasonality.png', bbox_inches='tight')
plt.show()
print('✅ Chart 5 saved')

In [ ]:
# ── Chart 6: Return Distribution Box Plot ─────────────────────────────────
sector_order = stock_summary.groupby('Sector')['ann_return'].mean().sort_values(ascending=False).index
fig, ax = plt.subplots(figsize=(13,6))
sns.boxplot(data=returns_long, x='Sector', y='DailyReturn', order=sector_order,
            palette='muted', flierprops={'marker':'o','markersize':2,'alpha':0.3}, ax=ax)
ax.axhline(0, color='black', linewidth=0.8, linestyle='--')
ax.yaxis.set_major_formatter(mtick.PercentFormatter(xmax=1, decimals=1))
ax.set_title('Daily Return Distribution by Sector (2020–2025)', **TITLE_KW)
ax.set_xlabel('Sector'); ax.set_ylabel('Daily Return')
plt.tight_layout()
plt.savefig('plots/plot_06_return_distribution.png', bbox_inches='tight')
plt.show()
print('✅ Chart 6 saved')

In [ ]:
# ── Chart 7: Rolling 30-Day Volatility ───────────────────────────────────
rolling_vol = sector_daily.rolling(30).std() * np.sqrt(252) * 100
fig, ax = plt.subplots(figsize=(14,6))
for col in rolling_vol.columns:
    ax.plot(rolling_vol.index, rolling_vol[col], label=col, linewidth=1.8,
            alpha=0.85, color=sector_colors[col])
ax.yaxis.set_major_formatter(mtick.PercentFormatter())
ax.set_title('Rolling 30-Day Annualised Volatility by Sector (2020–2025)', **TITLE_KW)
ax.set_xlabel('Date'); ax.set_ylabel('Annualised Volatility (%)')
ax.legend(title='Sector', bbox_to_anchor=(1.01,1), loc='upper left')
plt.tight_layout()
plt.savefig('plots/plot_07_rolling_volatility.png', bbox_inches='tight')
plt.show()
print('✅ Chart 7 saved')

## Section 6 — Predictive Model: 2026 Sector Forecasts

In [ ]:
# ── Linear Trend Model with Bootstrap Confidence Intervals ───────────────
# Method: Fit a linear regression (year vs annual return) per sector,
# then predict year 6 (2026). Bootstrap 500 resamples for CI.
# Why linear regression? With only 5-6 annual data points, simpler models
# are more robust and easier to interpret — a key DA principle.

rec_colors = {'OVERWEIGHT':'#2166ac','NEUTRAL':'#f4a460','UNDERWEIGHT':'#d73027'}
predictions_2026 = {}

for sector in SECTORS.keys():
    df = sector_yearly[sector_yearly['Sector'] == sector].copy()
    X  = (df['Year'] - 2020).values.reshape(-1, 1)
    y  = df['yearly_return_pct'].values

    model = LinearRegression().fit(X, y)
    pred  = model.predict([[6]])[0]

    # Bootstrap CI
    boot_preds = []
    for _ in range(500):
        idx = np.random.choice(len(X), len(X), replace=True)
        m2  = LinearRegression().fit(X[idx], y[idx])
        boot_preds.append(m2.predict([[6]])[0])

    ci_l = min(np.percentile(boot_preds, 10), pred)
    ci_u = max(np.percentile(boot_preds, 90), pred)
    mae  = mean_absolute_error(y, model.predict(X))
    r2   = r2_score(y, model.predict(X))

    predictions_2026[sector] = {
        'predicted_return_pct': round(pred, 1),
        'ci_lower':             round(ci_l, 1),
        'ci_upper':             round(ci_u, 1),
        'trend_slope_pct':      round(model.coef_[0], 2),
        'model_r2':             round(r2, 3),
        'model_mae_pct':        round(mae, 2),
        'recommendation':       'OVERWEIGHT' if pred > 10 else ('NEUTRAL' if pred > 0 else 'UNDERWEIGHT')
    }

pred_df = pd.DataFrame(predictions_2026).T.reset_index().rename(columns={'index':'Sector'})
pred_df = pred_df.sort_values('predicted_return_pct', ascending=False).reset_index(drop=True)

print('=' * 65)
print('  2026 SECTOR INVESTMENT OUTLOOK')
print('=' * 65)
for _, row in pred_df.iterrows():
    arrow = '▲' if float(row['predicted_return_pct']) > 0 else '▼'
    print(f"  {row['Sector']:12s}  {arrow} {float(row['predicted_return_pct']):+.1f}%"
          f"  CI:[{float(row['ci_lower']):+.1f}%, {float(row['ci_upper']):+.1f}%]"
          f"  → {row['recommendation']}")
print('=' * 65)
print('\n⚠️  Disclaimer: These forecasts are based on linear trend extrapolation')
print('   from 5 years of historical data. Past performance does not guarantee')
print('   future results. Not financial advice.')
pred_df

In [ ]:
# ── Chart 8: 2026 Forecast Bar Chart ─────────────────────────────────────
pred_sorted  = pred_df.sort_values('predicted_return_pct', ascending=True)
bar_colors_p = [rec_colors[r] for r in pred_sorted['recommendation']]
xerr_lower   = np.maximum(pred_sorted['predicted_return_pct'].astype(float) - pred_sorted['ci_lower'].astype(float), 0)
xerr_upper   = np.maximum(pred_sorted['ci_upper'].astype(float) - pred_sorted['predicted_return_pct'].astype(float), 0)

fig, ax = plt.subplots(figsize=(12,6))
bars = ax.barh(pred_sorted['Sector'], pred_sorted['predicted_return_pct'].astype(float),
               color=bar_colors_p, edgecolor='white', height=0.55)
ax.errorbar(pred_sorted['predicted_return_pct'].astype(float), pred_sorted['Sector'],
            xerr=[xerr_lower.values, xerr_upper.values],
            fmt='none', color='black', capsize=6, linewidth=1.5)
ax.axvline(0, color='black', linewidth=0.9)
ax.bar_label(bars, labels=[f"{v:.1f}%" for v in pred_sorted['predicted_return_pct'].astype(float)],
             padding=6, fontsize=10)
legend_elements = [Patch(facecolor='#2166ac', label='OVERWEIGHT  (>10% forecast)'),
                   Patch(facecolor='#f4a460', label='NEUTRAL  (0–10% forecast)'),
                   Patch(facecolor='#d73027', label='UNDERWEIGHT  (<0% forecast)')]
ax.legend(handles=legend_elements, loc='lower right', fontsize=10)
ax.set_title('2026 Sector Return Forecast — Linear Trend Model\n(with 80% Bootstrap Confidence Interval)', **TITLE_KW)
ax.set_xlabel('Predicted Annual Return (%)')
ax.xaxis.set_major_formatter(mtick.PercentFormatter())
plt.tight_layout()
plt.savefig('plots/plot_08_2026_prediction.png', bbox_inches='tight')
plt.show()
print('✅ Chart 8 saved')

In [ ]:
# ── Chart 9: Historical + 2026 Forecast per Sector (3x2 Grid) ────────────
hist_pivot = sector_yearly.pivot(index='Year', columns='Sector', values='yearly_return_pct')
fig, axes  = plt.subplots(2, 3, figsize=(16, 9))
axes = axes.flatten()

for i, sector in enumerate(SECTORS.keys()):
    ax  = axes[i]
    col = sector_colors[sector]
    ax.bar(hist_pivot.index, hist_pivot[sector], color=col, alpha=0.7, label='Actual', width=0.5)
    pv  = float(pred_df.loc[pred_df['Sector']==sector, 'predicted_return_pct'].values[0])
    cl  = float(pred_df.loc[pred_df['Sector']==sector, 'ci_lower'].values[0])
    cu  = float(pred_df.loc[pred_df['Sector']==sector, 'ci_upper'].values[0])
    yl  = max(pv - cl, 0)
    yu  = max(cu - pv, 0)
    ax.bar([2026], [pv], color=col, alpha=1.0, label='2026 Forecast', width=0.5,
           edgecolor='black', linewidth=1.5)
    ax.errorbar([2026], [pv], yerr=[[yl],[yu]], fmt='none', color='black', capsize=6, linewidth=2)
    ax.axhline(0, color='black', linewidth=0.7, linestyle='--')
    ax.set_title(sector, fontsize=12, fontweight='bold')
    ax.set_xlabel('Year'); ax.set_ylabel('Annual Return (%)')
    ax.yaxis.set_major_formatter(mtick.PercentFormatter())
    rec = pred_df.loc[pred_df['Sector']==sector, 'recommendation'].values[0]
    rc  = rec_colors[rec]
    ax.text(0.97, 0.95, rec, transform=ax.transAxes, ha='right', va='top',
            fontsize=9, fontweight='bold', color=rc,
            bbox=dict(boxstyle='round,pad=0.3', facecolor='white', edgecolor=rc))

handles, labels = axes[0].get_legend_handles_labels()
fig.legend(handles, labels, loc='lower center', ncol=2, fontsize=11, bbox_to_anchor=(0.5, -0.02))
fig.suptitle('Historical Annual Returns (2020–2025) + 2026 Sector Forecasts',
             fontsize=15, fontweight='bold', y=1.01)
plt.tight_layout()
plt.savefig('plots/plot_09_historical_vs_forecast.png', bbox_inches='tight')
plt.show()
print('✅ Chart 9 saved — all 9 charts complete')

## Section 7 — Export for Tableau & Summary

In [ ]:
# ── Export all CSVs ───────────────────────────────────────────────────────
returns_long.to_csv('data/tableau_daily_returns.csv', index=False)
stock_summary.to_csv('data/tableau_stock_summary.csv', index=False)
sector_cumret.reset_index().melt(id_vars='Date', var_name='Sector', value_name='CumulativeReturn') \
    .to_csv('data/tableau_sector_cumret.csv', index=False)
returns_long.groupby(['Year','Month'])['DailyReturn'].sum().reset_index() \
    .rename(columns={'DailyReturn':'MonthlyReturn'}) \
    .to_csv('data/tableau_seasonality.csv', index=False)
pred_df.to_csv('data/tableau_2026_predictions.csv', index=False)
sector_yearly.to_csv('data/tableau_sector_yearly.csv', index=False)

print('✅ Exported 6 CSV files to data/ folder')
print()

# ── Final Summary ──────────────────────────────────────────────────────────
best_sector   = stock_summary.groupby('Sector')['ann_return'].mean().idxmax()
best_sharpe   = stock_summary.groupby('Sector')['sharpe'].mean().idxmax()
best_stock    = stock_summary.loc[stock_summary['ann_return'].idxmax(), 'Ticker']
ow_sectors    = pred_df[pred_df['recommendation']=='OVERWEIGHT']['Sector'].tolist()
uw_sectors    = pred_df[pred_df['recommendation']=='UNDERWEIGHT']['Sector'].tolist()

print('=' * 55)
print('  FINAL FINDINGS SUMMARY')
print('=' * 55)
print(f'  Best 5yr sector (return) : {best_sector}')
print(f'  Best sector (Sharpe)     : {best_sharpe}')
print(f'  Top individual stock     : {best_stock}')
print(f'  2026 OVERWEIGHT sectors  : {", ".join(ow_sectors)}')
print(f'  2026 UNDERWEIGHT sectors : {", ".join(uw_sectors) if uw_sectors else "None"}')
print('=' * 55)
print()
print('Project complete. Files saved to:')
print('  plots/  — 9 PNG charts')
print('  data/   — 6 CSVs (for Tableau) + SQLite DB')